## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Шаг 1. Откроем файл с данными и изучим общую информацию. 

Импорт библиотек

In [1]:
import pandas as pd # <импорт библиотеки pandas>

Прочитаем файл datasets/data.csv и сохраним его в переменной data

In [2]:
data=pd.read_csv('/datasets/data.csv') # чтение файла с данными с сохранением в data

Получение первых 10 строк таблицы.

In [3]:
data.head(10) # <получение первых 10 строк таблицы data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Общая информация о данных таблицы.

In [4]:
data.info() #получение общей информации о данных в таблице data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Рассмотрим полученную информацию подробнее.
Всего в таблице 12 столбцов, тип данных у каждого столбца - строка.
Подробно разберём, какие в *data* столбцы и какую информацию они содержат:

* children — количество детей в семье

* days_employed — общий трудовой стаж в днях

* dob_years — возраст клиента в годах

* education — уровень образования клиента

* education_id — идентификатор уровня образования

* family_status — семейное положение

* family_status_id — идентификатор семейного положения

* gender — пол клиента

* income_type — тип занятости

* debt — имел ли задолженность по возврату кредитов

* total_income — ежемесячный доход

* purpose — цель получения кредита

Количество значений в столбцах различается. Это говорит о том, что в данных есть пустые значения.

### Вывод 

Каждая строка таблицы содержит информацию о клиенте.
Выявлены проблемы с пустыми строками, регистром значений столбца образование. Столбцы education — уровень образования клиента и family_status — семейное положение имеют дублирующие столбцы education_id — идентификатор уровня образования и family_status_id — идентификатор семейного положения, в качестве идентификатора принимают числовые значения. 

### Шаг 2. Предобработка данных

### Обработка пропусков

Проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений.

In [5]:
data.isnull().sum()# <суммарное количество пропусков, выявленных методом isnull() 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Т.к. количество пропусков в столбце days_employed и total_income совпадают,проверим гипотезу, что клиенты без стажа работы не имеют дохода, а значит пропуски не случайны. Для этого убедимся, что столбцы содержащие пропуски в столбце days_employed, также имеют пропуск в столбце total_income.

In [6]:
data[data["days_employed"].isnull() & data["total_income"].isnull()].count()# <проверка>

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

In [7]:
income_type_null_line=data[data['total_income'].isnull()]['income_type'].unique()
income_type_null_line

array(['пенсионер', 'госслужащий', 'компаньон', 'сотрудник',
       'предприниматель'], dtype=object)

In [8]:
print('Количество отрицательных значений в столбце "общий трудовой стаж в днях":',len(data[data['days_employed'] < 0]))
print('Количество отрицательных значений в столбце "ежемесячный доход":',len(data[data['total_income'] < 0]))

Количество отрицательных значений в столбце "общий трудовой стаж в днях": 15906
Количество отрицательных значений в столбце "ежемесячный доход": 0


In [9]:
data.loc[data['days_employed'] <0, 'days_employed']=-1*(data.loc[data['days_employed'] <0, 'days_employed'])

In [27]:
decribing_median=[] # переменная для хранения медианых значений
def decribing(income_type_null_line,column): # функция для вывода основных статистических значений
    decribing=data[data['income_type']==income_type_null_line][column].describe()
    print('˅'*120)
    print('Основные статистические характеристики столбца "',column,'" по типу занятости "',income_type_null_line,'"\n',decribing)
    print('*'*120)
    print('Медианное значение столбца "',column,'" по типу занятости "',income_type_null_line,'" равно {:.2f}'.format(decribing[5]))
    print('Среднее  значение  столбца "',column,'" по типу занятости "',income_type_null_line,'" равно {:.2f}'.format(decribing[1]))
    print('Среднее > Медианного значения столбца "',column,'" по типу занятости "',income_type_null_line,'" больше на {:.2f}'.format(decribing[1]-decribing[5]))
    print('^'*120,'\n')
    return decribing_median.append(decribing[5])
       
for i in range(len(income_type_null_line)): # цикл прогоняет все значения из income_type_null_line
    decribing(income_type_null_line[i],'total_income')

˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅
Основные статистические характеристики столбца " total_income " по типу занятости " пенсионер "
 count      3443.000000
mean     137127.465690
std       80246.953231
min       20667.263793
25%       82881.443465
50%      118514.486412
75%      169700.433010
max      735103.270167
Name: total_income, dtype: float64
************************************************************************************************************************
Медианное значение столбца " total_income " по типу занятости " пенсионер " равно 118514.49
Среднее  значение  столбца " total_income " по типу занятости " пенсионер " равно 137127.47
Среднее > Медианного значения столбца " total_income " по типу занятости " пенсионер " больше на 18612.98
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ 

˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅

In [28]:
median_total_income_dictionary={} #словарь хранит медиану по типу занятости
for i in range(len(income_type_null_line)):
    median_total_income_dictionary[income_type_null_line[i]]=decribing_median[i]

median_total_income_dictionary   

{'пенсионер': 118514.48641164352,
 'госслужащий': 150447.9352830068,
 'компаньон': 172357.95096577113,
 'сотрудник': 142594.39684740017,
 'предприниматель': 499163.1449470857}

In [29]:
def zamena_null_on_median(column,income_type_null_line,median):
    data.loc[data['income_type'] == income_type_null_line, column]=data.loc[data['income_type'] == income_type_null_line, column].fillna(median)
    return
for i in range(len(income_type_null_line)):
    zamena_null_on_median('total_income',income_type_null_line[i],median_total_income_dictionary[income_type_null_line[i]])   

In [30]:
decribing_median=[] #очистим предыдущие медианные значения
for i in range(len(income_type_null_line)):
    decribing(income_type_null_line[i],'days_employed')

˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅
Основные статистические характеристики столбца " days_employed " по типу занятости " пенсионер "
 count      3443.000000
mean     365003.491245
std       21069.606065
min      328728.720605
25%      346649.346146
50%      365213.306266
75%      383231.396871
max      401755.400475
Name: days_employed, dtype: float64
************************************************************************************************************************
Медианное значение столбца " days_employed " по типу занятости " пенсионер " равно 365213.31
Среднее  значение  столбца " days_employed " по типу занятости " пенсионер " равно 365003.49
Среднее > Медианного значения столбца " days_employed " по типу занятости " пенсионер " больше на -209.82
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ 

˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅

In [31]:
median_days_employed_dictionary={} #словарь медианных значений стажа для типа занятости
for i in range(len(income_type_null_line)):
    median_days_employed_dictionary[income_type_null_line[i]]=decribing_median[i]

median_days_employed_dictionary   

{'пенсионер': 365213.30626573117,
 'госслужащий': 2689.3683533043886,
 'компаньон': 1547.3822226779334,
 'сотрудник': 1574.2028211070851,
 'предприниматель': 520.8480834953765}

In [32]:
for i in range(len(income_type_null_line)):
    zamena_null_on_median('days_employed',income_type_null_line[i],median_days_employed_dictionary[income_type_null_line[i]])  

Проверим, остались ли пропуски в нашей таблице.

In [33]:
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Пропуски были обработаны и заменены на медианные значения в соответствие с типом дохода клиентов.Отсутствие пропусков в таблице позволяет производить количественные сравнения  переменных. Продолжим анализировать таблицу. 

### Замена типа данных

Столбцы общий трудовой стаж в днях 'days_employed' и ежемесячный доход 'total_income' имеют вещественный тип данных, заменим его на целочисленный. Для этого воспользуемся методом astype().


In [34]:
data['days_employed']=data['days_employed'].astype('int')
data['total_income']=data['total_income'].astype('int')

Убедимся, что замена типа данных прошла успешно, для этого воспользуемся методом info().

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Мы изменили тип данных для столбцов общий трудовой стаж в днях 'days_employed' и ежемесячный доход 'total_income' с вещественного на целочисленное.
Необходимо уточнить у программиста отвечающего за выгрузку, с чем могут быть связаны отрицательные и вещественные значения в столбце days_employed.Если это проблема с вводом данных, то необходимо отправить рекомендации о "заглушке" по вводу данных отрицательных и вещественных значений для столбца общий трудовой стаж в днях 'days_employed', т.к. количество отработанных дней не может быть отрицательным или дробным числом. Таким образом уменьшится количество входного "мусора".

### Обработка дубликатов

Проверим есть ли внашей таблице дубликаты, узнаем их количество.

In [36]:
data.duplicated().sum() #получение суммарного количества дубликатов в таблице data

54

Для выявления "мусора" связанного с написанием значений выедем на экран уникальные значения категориальных столбцов education, family_status, income_type, purpose. 

In [37]:
columns_for_check=['education', 'family_status', 'income_type', 'purpose']
for name_column in columns_for_check:
    print('Столбец ',name_column, '\n',data[name_column].unique(),'\n') #вывод уникальных значений 

Столбец  education 
 ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень'] 

Столбец  family_status 
 ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем'] 

Столбец  income_type 
 ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете'] 

Столбец  purpose 
 ['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижим

Видно что слобец 'education' имеет одинаковые значения записанные в разных регистрах. Переведем в нижний регистр значения с помощью метода str.lower() столбцов education для полного определения дубликатов в таблице data.

In [39]:
data['education']=data['education'].str.lower() #перевод в нижний регистр

Снова проверим столбец education на уникальность значений.

In [40]:
data['education'].unique() # уникальные значения

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Проверим еще раз на дубликаты.

In [41]:
data.duplicated().sum() # получение суммарного количества дубликатов в таблице data

71

Удалим дубликаты используя метод drop_duplicates().
При вызове метода drop_duplicates()
вместе с повторяющимися строками
удаляются их индексы, поэтому
используется с методом reset_index()

In [42]:
data=data.drop_duplicates().reset_index(drop=True) # удаление всех дубликатов из таблицы data специальным методом
data.duplicated().sum()

0

### Вывод

Мы избавились от дубликатов в таблице data. Приведение значений столбца education к нижнему регистру позволило обнаружить дополнительно 17дубликатов.

### Лемматизация

Посмотрим какие значения содержатся в столбце цель получения кредита 'purpose'.

In [43]:
#purpose=data['purpose'].unique()
data['purpose'].value_counts().reset_index()

,index,purpose
0,свадьба,791
1,на проведение свадьбы,768
2,сыграть свадьбу,765
3,операции с недвижимостью,675
4,покупка коммерческой недвижимости,661
5,операции с жильем,652
6,покупка жилья для сдачи,651
7,операции с коммерческой недвижимостью,650
8,покупка жилья,646
9,жилье,646


Лемматизируем значения столбца цель получения кредита 'purpose' с помощью инструмента Mystem библиотеки pymystem3. Создадим новый столбец 'purpose_lem' с лемматинизированными значениями столбца цель получения кредита 'purpose', затем выведемего на экран.

In [44]:
from pymystem3 import Mystem
m = Mystem()
data['purpose_lem']=data['purpose'] #создадим столбец data['purpose_lem'] путем копирования столбца data['purpose']
#создадим столбец с лемманитизированным значением столбца purpose
for i in range(len(data)):
    data['purpose_lem'][i]=m.lemmatize(data['purpose'][i])
data['purpose_lem'].value_counts().reset_index()                           

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,index,purpose_lem
0,"[автомобиль, \n]",972
1,"[свадьба, \n]",791
2,"[на, , проведение, , свадьба, \n]",768
3,"[сыграть, , свадьба, \n]",765
4,"[операция, , с, , недвижимость, \n]",675
5,"[покупка, , коммерческий, , недвижимость, \n]",661
6,"[операция, , с, , жилье, \n]",652
7,"[покупка, , жилье, , для, , сдача, \n]",651
8,"[операция, , с, , коммерческий, , недвижимо...",650
9,"[покупка, , жилье, \n]",646


### Вывод

### Категоризация данных

In [45]:
data['purpose_category']=data['purpose'] #создадим столбец data['purpose_category'] путем копирования столбца data['purpose']
purpose_lem_list=['автомобиль','свадьба','недвижимость','жилье', 'образование']
def change_purpose(name,i): #функция по заполнению столбца data['purpose_category']
    if 'жилье' in data['purpose_lem'][i]:
        data['purpose_category'][i]='недвижимость'
    elif 'недвижимость' in data['purpose_lem'][i]:
        data['purpose_category'][i]='недвижимость'
    elif 'автомобиль' in data['purpose_lem'][i]:
        data['purpose_category'][i]='автомобиль'
    elif 'образование' in data['purpose_lem'][i]:
        data['purpose_category'][i]='образование'
    elif 'свадьба' in data['purpose_lem'][i]:
        data['purpose_category'][i]='свадьба'
a=-1
while a<5:
    a+=1
    if a == 5:
        break
    for i in range(len(data['purpose_lem'])):
        change_purpose(purpose_lem_list[a],i)
    

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.7

Проверим значения столбца 'purpose_category'.

In [46]:
data['purpose_category'].unique()

array(['недвижимость', 'автомобиль', 'образование', 'свадьба'],
      dtype=object)

### Вывод

Категоризация данных упрощает работу по анализу данных, 38 значений цели кредита мы свели к 4 группам.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Проверим на отрицательные значения столбец 'children'.

In [47]:
print('Количество отрицательных значений в столбце количество детей в семье :',len(data[data['children'] < 0]))

Количество отрицательных значений в столбце количество детей в семье : 47


Исправим отрицательные значения столбца 'children',затем провдем проверку.

In [48]:
data.loc[data['children'] <0, 'children']=-1*(data.loc[data['children'] <0, 'children'])

In [49]:
print('Количество отрицательных значений в столбце количество детей в семье :',len(data[data['children'] < 0]))

Количество отрицательных значений в столбце количество детей в семье : 0


Чтобы ответить на поставленный вопрос необходимо сформировать сводную таблицу по столбцу количество детей 'children' и наличие задолжности по кредиту 'debt'. Затем произведем расчеты.

In [50]:
#сформируем сводную таблицу кол-во клиентов в соответствии со столбцом кол-во детей
data_pivot_children_count = data.pivot_table(index=['children'], values='debt', aggfunc='count')
#сформируем сводную таблицу клиентов имеющих задолжность в соответствии со столбцом кол-во детей
data_pivot_children_sum = data.pivot_table(index=['children'], values='debt', aggfunc='sum')
#сведем две таблицы вместе
data_pivot_children = data_pivot_children_count.merge(data_pivot_children_sum, on='children', how='left')
#переименуем название столбцов
data_pivot_children.rename(columns={'debt_x': 'sum_of_clients', 'debt_y': 'sum_of_debt'}, inplace=True)
data_pivot_children #выведем таблицу

,sum_of_clients,sum_of_debt
children,,
0,14091,1063
1,4855,445
2,2052,194
3,330,27
4,41,4
5,9,0
20,76,8


Произведем расчеты распределения долей клиентов с/без детей среди потребителей и должников по кредитам.

In [51]:
data_pivot_children['sum_of_clients'].sum()     #всего клиентов 
data_pivot_children['sum_of_clients'][0]        #клиенты без детей
data_pivot_children['sum_of_clients'][1:].sum() #клиенты с детьми 
data_pivot_children['sum_of_debt'].sum()        #клиенты имевшие задолжность 
data_pivot_children['sum_of_debt'][0]           #клиенты без детей клиенты имевшие задолжность 
data_pivot_children['sum_of_debt'][1:].sum()    #клиенты с детьми имевшие задолжность 
print('Доля клиентов без детей относительно всех клиентов составляет {:.1%}'.format(data_pivot_children['sum_of_clients'][0] / data_pivot_children['sum_of_clients'].sum()))
print('Доля клиентов без детей имевшие проблемы с возвратом  кредита составляет {:.1%}'.format(data_pivot_children['sum_of_debt'][0] / data_pivot_children['sum_of_clients'][0]))
print('Доля клиентов с детьми имевшие проблемы с возвратом  кредита составляет {:.1%}'.format(data_pivot_children['sum_of_debt'][1:].sum() / data_pivot_children['sum_of_clients'][1:].sum()))

Доля клиентов без детей относительно всех клиентов составляет 65.7%
Доля клиентов без детей имевшие проблемы с возвратом  кредита составляет 7.5%
Доля клиентов с детьми имевшие проблемы с возвратом  кредита составляет 9.2%


### Вывод

Проблемы с возвратом кредитов имеют 9,2% клиентов с детьми, в то время как доля клиентов без детей составляет 7,5%.
Клиенты не имеющие детей чаще пользуются услугами кредитования их доля составляет 65.7%, статистика по возврату кредита в срок для данной группы по сравнению с группой имеющие детей выше на 1,7%.
Вывод: дети не только цветы жизни, но и головная боль=)

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для ответа на вопрос создадим сводную таблицу data_pivot_family_status

In [52]:
#действия произведем по аналогии
data_pivot_family_status_count = data.pivot_table(index=['family_status'], values='debt', aggfunc='count')
data_pivot_family_status_sum = data.pivot_table(index=['family_status'], values='debt', aggfunc='sum')
data_pivot_family_status = data_pivot_family_status_count.merge(data_pivot_family_status_sum, on='family_status', how='left')
data_pivot_family_status['доли должников в %'] =data_pivot_family_status_sum/data.pivot_table(index=['family_status'], values='debt', aggfunc='count')*100
data_pivot_family_status['доли должников в %']=data_pivot_family_status['доли должников в %'].astype('int')
data_pivot_family_status.rename(columns={'debt_x': 'sum_of_clients', 'debt_y': 'sum_of_debt'}, inplace=True)
data_pivot_family_status

,sum_of_clients,sum_of_debt,доли должников в %
family_status,,,
Не женат / не замужем,2810,274,9
в разводе,1195,85,7
вдовец / вдова,959,63,6
гражданский брак,4151,388,9
женат / замужем,12339,931,7


### Вывод

Наилучший показатель по возвратам кредитов является у клиентов со статусом "вдовец / вдова", лишь 6% из данной группы имели задолжность.
На втором месте идет группа людей со статусами "в разводе" и "женат / замужем", 7% клиентов имели проблемы с возвратом кредита.
На третьем месте оказалась группа людей со статусами "Не женат / не замужем" и "гражданский брак" с 9%.		

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Разделим уровень дохода на три категории "низкий доход", "средний доход" и "высокий доход", для определения пороговых значений категорий воспользуемся функцией describe()

In [53]:
print(data['total_income'].describe(),'\n')
low_income=(data['total_income'].describe()[4])
high_income=data['total_income'].describe()[6]
print('Пороговое значение для категории  "низкий доход" составляет: {:.2f}'.format(low_income))
print('Пороговое значение для категории "высокий доход" составляет: {:.2f}'.format(high_income))

count    2.145400e+04
mean     1.653196e+05
std      9.818730e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.425940e+05
75%      1.958202e+05
max      2.265604e+06
Name: total_income, dtype: float64 

Пороговое значение для категории  "низкий доход" составляет: 107623.00
Пороговое значение для категории "высокий доход" составляет: 195820.25


Создадим столбец с уровнем дохода назовем его 'lvl_of_income'

In [54]:
data['lvl_of_income']=data['total_income'] # создаем столбец data['lvl_of_income'] путем копрования
def make_lvl_of_income(income,i):          # функция по заполнению столбца data['lvl_of_income']
    if income <= low_income:
        data['lvl_of_income'][i]='низкий доход'
    elif low_income < income < high_income:
        data['lvl_of_income'][i]='средний доход'
    else:
        data['lvl_of_income'][i]='высокий доход'
for i in range(len(data['total_income'])):
    make_lvl_of_income(data['total_income'][i],i)
    

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4:

In [55]:
pivot_lvl_income_count = data.pivot_table(index=['lvl_of_income'], values='debt',aggfunc='count')  
pivot_lvl_income_sum = data.pivot_table(index=['lvl_of_income'], values='debt',aggfunc='sum')   
pivot_lvl_income = pivot_lvl_income_count.merge(pivot_lvl_income_sum, on='lvl_of_income', how='left')
pivot_lvl_income['доли должников в %']=pivot_lvl_income_sum/pivot_lvl_income_count*100
pivot_lvl_income.rename(columns={'debt_x': 'sum_of_clients', 'debt_y': 'sum_of_debt'}, inplace=True)
pivot_lvl_income

,sum_of_clients,sum_of_debt,доли должников в %
lvl_of_income,,,
высокий доход,5364,383,7.140194
низкий доход,5364,427,7.960477
средний доход,10726,931,8.679843


### Вывод

Лучший показатель по возврату кредита имеют клиенты с высоким уровнем дохода.
7,1% должников имеют высокий доход, против 8% с низким.
Наибольшая часть кредитов выдана клиентам со среднем уровнем дохода, должников среди них 8,7%

- Как разные цели кредита влияют на его возврат в срок?

In [56]:
data_pivot_purpose_count = data.pivot_table(index=['purpose_category'], values='debt',aggfunc='count') 
data_pivot_purpose_sum = data.pivot_table(index=['purpose_category'], values='debt',aggfunc='sum') 
data_pivot_purpose=data_pivot_purpose_count.merge(data_pivot_purpose_sum, on='purpose_category', how='left')
data_pivot_purpose['доли должников в %']=data_pivot_purpose_sum / data_pivot_purpose_count*100
data_pivot_purpose['доля кредитов по целям в %']= data_pivot_purpose_count/data_pivot_purpose_count[:].sum()*100
data_pivot_purpose.rename(columns={'debt_x': 'sum_of_clients', 'debt_y': 'sum_of_debt'}, inplace=True)
data_pivot_purpose

,sum_of_clients,sum_of_debt,доли должников в %,доля кредитов по целям в %
purpose_category,,,,
автомобиль,4306,403,9.359034,20.070849
недвижимость,10811,782,7.233373,50.391535
образование,4013,370,9.220035,18.705137
свадьба,2324,186,8.003442,10.832479


### Вывод

Половина кредитов выдана для покупки недвижимости, также для этой цели наивысший показатель по возврату кредита.
7,2% клиентов имели долг по возврату кредита.
Количество выданных кредитов для целей покупки автомобиля и получения образования,  и составляют 20,1% и 18,7% соответственно. Доля должников для данных категорий практичестки одинаковы 9,4% -автомобиль, 9,2%-образование.
Клиенты берущие кредит на свадьбу составляет 10%, в то время как доля должников среди них 8%.

### Шаг 4. Общий вывод

Для составления скорринговой модели необходимо учесть полученные результаты анализа. 
В новой модели скорринга необходимо увеличить показатель для клиентов из групп:
* целью покупки которых является недвижимость
* вдовец/вдова
* с высоким уровнем дохода
* клиенты без детей